## <center> **Credit Card Fraud (Anomaly/Outlier) Detection** 
<center><em>The processes of determining an entry among entries that does not seem to belong. In this case we are using the Anonymized Credit Card dataset from Kaggle, which has  transactions labeled as fraudulent or genuine to create an ML Fraud Detection model.</em></center>
<br>
<center><img src="Assets/CCFD.png" width=600/></center>

***Project Contributors:*** Kuzi Rusere<br>
**MBA streamlit App URL:** N/A